In [16]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    get_region_mapping_base,
)
from polars import testing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
region.Federal2022

electoralyze.region.regions.federal_2022.Federal2022

In [18]:
region.SA1_2021

electoralyze.region.regions.SA1_2021.SA1_2021

In [19]:
region.Federal2022.geometry

federal_2022,geometry
str,binary
"""Adelaide""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00W\x00\x00\x00u\x8e\x01\xd9kTa@\x82\xfd\xd7\xb9ioA\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xd0\x98I\xd4\x0bTa@\x0a\xd68\x9b\x8epA\xc0\x00\x00\x00""…"
"""Aston""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\xd0\x02\x00\x00\xfc\x8e\xe1\xb1\x1f+b@|\xed\x99%\x01\xeeB\xc0\x00\x00\x00\x00\x00\x00\x00\x00;8\xd8\x9b\x18+b@-\xe8\xbd1\x04\xeeB\xc0\x00\x00\x00""…"
"""Ballarat""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00D\x09\x00\x00J\xb2\x0eGW\xfaa@p\xb071$\x07C\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x05\xc0x\x06\x8d\xfaa@\xfa\xd1p\xca\xdc\x02C\xc0\x00\x00\x00""…"
"""Banks""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\x9b\x00\x00\x00\xab\x07\xccC&\xe4b@\x96\xaf\xcb\xf0\x9f\xfc@\xc0\x00\x00\x00\x00\x00\x00\x00\x006#\x83\xdc\xc5\xe3b@\xfc\xdf\x11\x15\xaa\xfd@\xc0\x00\x00\x00""…"
"""Barker""","b""\x01\x06\x00\x00\xa0\xe6\x10\x00\x00\x1b\x00\x00\x00\x01\x03\x00\x00\x80\x01\x00\x00\x00M\x0d\x00\x00wj.7\x18\xa0a@\xa3\x05h[\xcd\x02A\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xa0PO\x1f\x01\xa0a@r7""…"
…,…
"""Werriwa""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\x1d\x01\x00\x00*\x8d\x98\xd9g\xddb@\xf7\xe67L4\xf8@\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x1a\x17\x0e\x84d\xddb@G\xc8@\x9e]\xf8@\xc0\x00\x00\x00""…"
"""Whitlam""","b""\x01\x06\x00\x00\xa0\xe6\x10\x00\x00\x02\x00\x00\x00\x01\x03\x00\x00\x80\x01\x00\x00\x00\xd9\x04\x00\x00y\x95\xb5M\xf1\xdcb@\x17a\x8ariLA\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xaf\xb1KT\xef\xdcb@u\x1e""…"
"""Wide Bay""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00d\x0e\x00\x00\x8b\xe1\xea\x00\x88+c@\x01L\x198\xa0\x019\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xf7\xb1\x82\xdf\x86+c@6\x93o\xb6\xb9\x019\xc0\x00\x00\x00""…"


In [20]:
region.SA1_2021.geometry.head(100)

SA1_2021,geometry
i64,binary
10102100701,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\xc9\x02\x00\x00]\x16V\xf6\x80\xbcb@`j""\xe5\x88\x8bA\xc0K\x07\x81\xbe\x89\xbcb@\x98\xe7[Du\x8bA\xc0\xdc\x8d\x1c;\x92\xbcb@\xa9X\x94""…"
10102100702,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x02\x00\x00\x00\x08\x01\x00\x00Y\xdc\xe9\xa6~\xb7b@\x9e\xfb\xe0\x81\x0c\xafA\xc0\xb5Q\xabKL\xb8b@y\x87\xa0!}\xafA\xc0\xdc\xeb1\x0bW\xb8b@A\xb6\x8d""…"
10102100703,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x1a\x00\x00\x00\x0f\xf0N\x9b\xf5\xb8b@\xd60\xf1\x91g\xb8A\xc0u\xc3C\xa7\xf2\xb8b@G8\xcb\xb4+\xb8A\xc0B6\xc1\xac\xfb\xb8b@^p\xe5""…"
10102100704,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x1e\x00\x00\x00\x95\x92\x10<}\xb9b@7J\xbb\x9f\x09\xb9A\xc0\xca1\xac\xee\x89\xb9b@j\x94\xfc\x020\xb8A\xc0RI\xbet\x8b\xb9b@\xc1\xe3\xb4""…"
10102100705,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00""\x00\x00\x00\xdc\xbc\x02\x88p\xb9b@\xfe\x05l#y\xb9A\xc0g\x10R\x1cr\xb9b@X.\x07\x8cy\xb9A\xc0\x93\xecfnt\xb9b@Pc\x1f""…"
…,…
10102101221,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x10\x00\x00\x005[\xf6\xe7\x8d\xa6b@\xcb\x87\xf1u\x05\xaeA\xc0\xbbC\x9a\xa8\x94\xa6b@\xb2K\xa5Z\xfb\xadA\xc0F\xfd\x19\xd2\x99\xa6b@\x03\xaf\xc8""…"
10102101222,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x0e\x00\x00\x00=\xdaAU\x91\xa6b@s_\xc0h)\xb1A\xc0\x0d`\xa0T\x94\xa6b@\xbe\xa4\xab\x17\xf1\xb0A\xc0\xc3\xae@\xbb\xbe\xa6b@@O\x8a""…"
10102101223,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x13\x00\x00\x00\x91\x96\xa2KN\xa6b@y:\\xa0\x8c\xb1A\xc0\x9a\xba\xda\xe8V\xa6b@\x02\x17\xdd@\xe1\xb0A\xc0\x00&\x14\xe0d\xa6b@r\xdd\xf7""…"


In [21]:
(
    region.Federal2022.geometry.join(
        region.SA1_2021.geometry.head(10000),
        how="cross",
    )
    .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
)

In [ ]:
(
    region.Federal2022.geometry.pipe(to_geopandas)
    .merge(
        region.SA1_2021.geometry.head(10000).pipe(to_geopandas),
        how="cross",
    )
    # .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    # .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
    .loc[lambda df: df["geometry_x"].overlaps(df["geometry_y"])]
    .assign(area=lambda df: df["geometry_x"].intersection(df["geometry_y"]).area)
)

In [ ]:
(
    region.Federal2022.geometry.pipe(to_geopandas)
    .merge(
        region.SA1_2021.geometry.head(100).pipe(to_geopandas),
        how="cross",
    )
    # .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    # .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
    .loc[lambda df: df["geometry_x"].overlaps(df["geometry_y"])]
    .assign(area=lambda df: df["geometry_x"].intersection(df["geometry_y"]).area)
)

In [ ]:
some_intersection = _create_intersection_area_mapping(
    geometry_from=region.Federal2022.geometry, geometry_to=region.SA1_2021.geometry.head(100)
)
some_intersection

In [ ]:
target_sa1 = 10103101401
target_sa1 = 10106154239
region.SA1_2021.geometry.filter(pl.col(region.SA1_2021.id) == target_sa1).select(st.geom("geometry").st.area())

In [ ]:
(
    region.SA1_2021.geometry.filter(pl.col(region.SA1_2021.id) == target_sa1)
    .join(
        region.Federal2022.geometry.filter(pl.col(region.Federal2022.id).is_in(["Eden-Monaro", "Hume"])),
        how="cross",
    )
    .select(st.geom("geometry").st.intersection(pl.col("geometry_right")))
    # .head(1)
    # .tail(1)
    .pipe(to_geopandas)
    .explore()
)

In [ ]:
some_intersection.filter(pl.col(region.SA1_2021.id) == target_sa1)

In [ ]:
_create_intersection_area_mapping(
    geometry_from=region.Federal2022.get_raw_geometry(), geometry_to=region.SA1_2021.get_raw_geometry().head(100)
)

In [ ]:
intersection = get_region_mapping_base(
    region_from=region.Federal2022,
    region_to=region.SA1_2021,
    mapping_method="intersection_area",
    redistribute_with_full=True,
    save_data=True,
    force_new=True,
)
intersection